In [7]:
import datetime
import pystac
import rioxarray
import gcsfs
import rasterio 
import pyproj
import tqdm

import os

# from shapely.geometry import Polygon
# import geopandas as gpd

from geopandas import GeoSeries
from rasterio.features import shapes
from shapely.geometry import shape, mapping, Polygon, MultiPolygon
from pystac import Catalog, Collection, Item, Asset

# Create STAC catalog from retiled images

In this notebook we illustrate how to create a STAC catalog for organizing the GeoTIFFs that will be produced during the retiling of the satellite imagery (either original images or mosaics). STAC catalogs entail the following elements: Catalog > Collection > Item (definitions given [here](https://stacspec.org/core.html)). In order to construct the catalog we use [PySTAC](https://pystac.readthedocs.io), which provides a representation of these elements as Python objects. For a PySTAC tutorial on how to construct a general-purpose catalog, have a look at this [tutorial](https://pystac.readthedocs.io/en/latest/tutorials/how-to-create-stac-catalogs.html).

In [165]:
# access to GCS
import ee
ee.Initialize()
ee.Authenticate()

Enter verification code:  4/1AdQt8qggfdyUJhVAZyQ6Zi2PT_e5Crhb_f1BRd8iM0obAahviS7kcJoH2BY



Successfully saved authorization token.


In [174]:
# read authentication credentials created by `gcloud`
google_account = 'izeboud.maaike'
gcs_token=f"/Users/tud500158/.config/gcloud/legacy_credentials/{google_account}@gmail.com/adc.json"
gcs_fs = gcsfs.GCSFileSystem(
    # token=f"/Users/fnattino/.config/gcloud/legacy_credentials/{google_account}@gmail.com/adc.json"
    token=gcs_token
)


In [181]:
# get bucket tile list

gcloud_dir = "gs://ee-data_export/S1_mosaic_2020-01-01_2020-02-02/S1_*.tif"

tiles = gcs_fs.glob(
    gcloud_dir
)
print(len(tiles))
tiles[:10]

313


['ee-data_export/S1_mosaic_2020-01-01_2020-02-02/S1_EW_mosaic_2020-01-01_2020-02-01_tile_0.tif',
 'ee-data_export/S1_mosaic_2020-01-01_2020-02-02/S1_EW_mosaic_2020-01-01_2020-02-01_tile_1.tif',
 'ee-data_export/S1_mosaic_2020-01-01_2020-02-02/S1_EW_mosaic_2020-01-01_2020-02-01_tile_10.tif',
 'ee-data_export/S1_mosaic_2020-01-01_2020-02-02/S1_EW_mosaic_2020-01-01_2020-02-01_tile_100.tif',
 'ee-data_export/S1_mosaic_2020-01-01_2020-02-02/S1_EW_mosaic_2020-01-01_2020-02-01_tile_101.tif',
 'ee-data_export/S1_mosaic_2020-01-01_2020-02-02/S1_EW_mosaic_2020-01-01_2020-02-01_tile_102.tif',
 'ee-data_export/S1_mosaic_2020-01-01_2020-02-02/S1_EW_mosaic_2020-01-01_2020-02-01_tile_103.tif',
 'ee-data_export/S1_mosaic_2020-01-01_2020-02-02/S1_EW_mosaic_2020-01-01_2020-02-01_tile_104.tif',
 'ee-data_export/S1_mosaic_2020-01-01_2020-02-02/S1_EW_mosaic_2020-01-01_2020-02-01_tile_105.tif',
 'ee-data_export/S1_mosaic_2020-01-01_2020-02-02/S1_EW_mosaic_2020-01-01_2020-02-01_tile_106.tif']

# Create catalog

In [182]:


# define id's for the STAC objects
# catalog_id = "ice-shelfs-antartica_retiled_S1"
catalog_id = "S1_mosaic"
collection_id = "2020-01-01_2020-02-01"
collection_descript = "Mosaic Sentinel-1 images for time span 2020-01-01/2020-02-01"

# catalog_id = "S1_mosaic"
catalog_descript = "Mosaic Sentinel-1 images generated using GEE"

asset_id = "HH"

# for a mosaic, use earliest/latest datetimes of original images
start_datetime = datetime.datetime.fromisoformat("2020-01-01")
end_datetime = datetime.datetime.fromisoformat("2020-02-01") 

webdav_url="https://webdav.grid.surfsara.nl:2880"
tile_url = "/pnfs/grid.sara.nl/data/iceshelf/disk/S1_mosaic_2020-01-01_2020-02-01/"

# tile = tiles[0]
# print(tile)
# _, filename = os.path.split(tile)
    
# source_uri = f"gs://{tile}"
# asset_uri= f"{webdav_url}{tile_url}{filename}"
# asset_uri
snellius_path = "/projects/0/einf512/S1_mosaic_2020-01-01_2020-02-01/"
asset_uri = f"{snellius_path}{filename}"
asset_uri

'/projects/0/einf512/S1_mosaic_2020-01-01_2020-02-01/S1_EW_mosaic_2020-01-01_2020-02-01_tile_99.tif'

In [183]:
items = {}

for tile in tqdm.tqdm(tiles):
    
    _, filename = os.path.split(tile)
    item_id, _ = os.path.splitext(filename)
    
    source_uri = f"gs://{tile}"
    # asset_uri= f"{webdav_url}{tile_url}{filename}" # dcache path
    asset_uri= f"{snellius_path}{filename}" # snellius path

    # read info from file in gcs
    with gcs_fs.open(source_uri) as f_read:
        # print('opened ', source_uri)

        
        with rasterio.Env(GOOGLE_APPLICATION_CREDENTIALS=f"{gcs_token}"):
            with rasterio.open('{}'.format(source_uri)) as src:
                # print(src.profile, src.crs, src.transform)

                # transfrom boundingbox
                left,bottom,right,top=src.bounds
                bbox = rasterio.warp.transform_bounds(src.crs, "WGS84", left, bottom, right, top )

                # create geometries
                poly_geom = Polygon.from_bounds(*src.bounds)
                polygons = GeoSeries(poly_geom, crs=src.crs ).to_crs("WGS84") 
                geometry = mapping(polygons) if len(polygons)>1 else mapping(polygons[0])
                # print(polygons.bounds)

                # Create Asset
                asset = Asset(
                    # href=source_uri,  # link to asset GCS
                    href=asset_uri,  # link to asset dCache
                    title=", ".join("HH"),
                    media_type=pystac.MediaType.GEOTIFF # or COG - verify e.g. with with https://github.com/rouault/cog_validator 
                )
                # asset

                # create Item object
                item = Item(
                    id=item_id,
                    geometry=geometry,
                    bbox=bbox,
                    datetime=None,  
                    properties=dict(
                        start_datetime=start_datetime.isoformat(),
                        end_datetime=end_datetime.isoformat()   
                    )
                )

                item.add_asset(asset_id, asset)

                items[item_id] = item        
                # print(item)
# items                

100%|███████████████████████████████████████████| 313/313 [00:22<00:00, 13.66it/s]


In [184]:
# spatial extent
footprints = (shape(i.geometry).envelope for i in items.values())
collection_bbox = MultiPolygon(footprints).bounds
spatial_extent = pystac.SpatialExtent(bboxes=[collection_bbox])

# temporal extent
start = (i.properties.get('start_datetime', item.datetime) for i in items.values())
start = sorted(start)[0]
end = (i.properties.get('end_datetime', item.datetime) for i in items.values())
end = sorted(end)[-1]
temporal_extent = pystac.TemporalExtent(
    intervals=[[datetime.datetime.fromisoformat(start), 
                datetime.datetime.fromisoformat(end)]]
)

extent = pystac.Extent(spatial=spatial_extent, temporal=temporal_extent)


# create Collection object

collection = Collection(
    id=collection_id,
    description=collection_descript,
    extent=extent,
)

collection.add_items(items.values())
collection


# create STAC catalog

catalog = Catalog(
    id=catalog_id,
    description=catalog_descript
)

catalog.add_child(collection)


# local save
catalog.normalize_and_save(f'./{catalog_id}', 'SELF_CONTAINED')

# catalog.describe()


# read tilepaths from catalog

In [171]:
import pathlib
import geopandas as gpd
import pandas as pd
def _read_tile_catalog(catalog_path):
    """ Read the tile catalog """
    catalog_path = pathlib.Path(catalog_path)
    catalog_path = catalog_path / "catalog.json"
    return pystac.Catalog.from_file(catalog_path.as_posix())

def _get_tile_paths(catalog, item_ids, asset_key=None):
    """ Extract the asset paths from the catalog """
    items = (catalog.get_item(item_id, recursive=True) for item_id in item_ids)
    if asset_key is None:
        item = list(catalog.get_all_items())[0]
        asset_key = list(item.assets.keys())[0]
    assets = (item.assets[asset_key] for item in items )       
    return [asset.get_absolute_href() for asset in assets]

def _catalog_to_geodataframe(catalog, crs="WGS84"):
    """ Convert STAC catalog to a GeoDataFrame object """
    features = {item.id: item.to_dict() for item in catalog.get_all_items()}
    gdf = gpd.GeoDataFrame.from_features(features.values())
    gdf.index = features.keys()
    for column in gdf.columns:
        if 'datetime' in column:
            gdf[column] = pd.to_datetime(gdf[column])
    gdf = gdf.set_crs(crs)
    return gdf


catalog = _read_tile_catalog('./S1_mosaic/')
# catalog = pystac.Catalog.from_file(os.path.join(homedir, 'Data/tiles/catalog.json'))
    

tiles = _catalog_to_geodataframe(catalog) # gpd f
tilelist = tiles.index.values.tolist()    # tilenames in list (excluding .tif)

# remove ROI tiles from tilelist, before dividing training/test/val
tileNums_test_ROI = [112,122,126,139,140,141,142,143,151,152,153,154] # to do: include in config file
tilelist_ROI = [item for item in tilelist if int(item.split('_')[-1]) in tileNums_test_ROI] 
# make SET from ROI tiles
ROI_set = tiles.loc[tilelist_ROI] # to do: include this in stored dataset.json 
# remove these from tilelist
tiles_set = tiles.index.difference(tilelist_ROI) 
tiles_set = tiles.loc[tiles_set]
tiles = tiles_set
tiles


# train_set_paths = _get_tile_paths(catalog, tiles.index, "B2-B3-B4-B11")



# item = list(catalog.get_all_items())[0]
# asset_id = list(item.assets.keys())[0]

# train_set_paths = _get_tile_paths(catalog, tiles.index, "HH")
train_set_paths = _get_tile_paths(catalog, tiles.index)#, asset_id)
train_set_paths

['https://webdav.grid.surfsara.nl:2880/pnfs/grid.sara.nl/data/iceshelf/disk/S1_mosaic_2020-01-01_2020-02-01/S1_EW_mosaic_2020-01-01_2020-02-01_tile_0.tif',
 'https://webdav.grid.surfsara.nl:2880/pnfs/grid.sara.nl/data/iceshelf/disk/S1_mosaic_2020-01-01_2020-02-01/S1_EW_mosaic_2020-01-01_2020-02-01_tile_1.tif',
 'https://webdav.grid.surfsara.nl:2880/pnfs/grid.sara.nl/data/iceshelf/disk/S1_mosaic_2020-01-01_2020-02-01/S1_EW_mosaic_2020-01-01_2020-02-01_tile_10.tif',
 'https://webdav.grid.surfsara.nl:2880/pnfs/grid.sara.nl/data/iceshelf/disk/S1_mosaic_2020-01-01_2020-02-01/S1_EW_mosaic_2020-01-01_2020-02-01_tile_100.tif',
 'https://webdav.grid.surfsara.nl:2880/pnfs/grid.sara.nl/data/iceshelf/disk/S1_mosaic_2020-01-01_2020-02-01/S1_EW_mosaic_2020-01-01_2020-02-01_tile_101.tif',
 'https://webdav.grid.surfsara.nl:2880/pnfs/grid.sara.nl/data/iceshelf/disk/S1_mosaic_2020-01-01_2020-02-01/S1_EW_mosaic_2020-01-01_2020-02-01_tile_102.tif',
 'https://webdav.grid.surfsara.nl:2880/pnfs/grid.sara.nl/

In [128]:
tiles.index
item_ids = tiles.index
items = (catalog.get_item(item_id, recursive=True) for item_id in item_ids)
# print(list(items))
item = list(items)[0]
asset_key = list(item.assets.keys())
print(asset_key)
bandname

asset_key = 'HH' # "B2-B3-B4-B11" #
assets = (item.assets[asset_key] for item in items)
assets
[asset.get_absolute_href() for asset in assets]

# B = (item.assets[asset_key] for item in items)
# B
# [list(x) for x in B]

['HH']


[]

In [139]:

homedir = '/Users/tud500158/Library/Mobile Documents/com~apple~CloudDocs/Documents/Documents - TUD500158/'
try:
    print("Hello")
except:
    print("Something went wrong")
else:
    print("Nothing went wrong")

Hello
Nothing went wrong


In [137]:
cat = catalog

# pystac.Catalog.from_file(catalog_path.as_posix())
catalog = pystac.Catalog.from_file('./S1_mosaic/catalog.json')
# catalog = pystac.Catalog.from_file(os.path.join(homedir, 'Data/tiles/catalog.json'))
cat



items = (catalog.get_item(item_id, recursive=True) for item_id in item_ids)
# print(list(items))
item = list(items)[0]
print(item)
list(item.assets.keys())

asset_id = list(item.assets.keys())[0]
print(asset_id)

assets = (item.assets[asset_id] for item in items)
print([asset.get_absolute_href() for asset in assets])

# features = {item.id: item.to_dict() for item in catalog.get_all_items()}
# features
# features.keys()


item = list(catalog.get_all_items())[0]
asset_id = list(item.assets.keys())[0]
print(asset_id)
assets = (item.assets["HH"] for item in items)
print([asset.get_absolute_href() for asset in assets])


<Item id=S1_EW_mosaic_2020-01-01_2020-02-01_tile_0>
HH
[]
HH
[]


In [62]:

# save it on dCache
# webdav_url="https://webdav.grid.surfsara.nl:2880"
# catalog.normalize_and_save(
#     f"{webdav_url}/pnfs/grid.sara.nl/data/iceshelf/disk/S1_mosaic_catalog", 
#     catalog_type="SELF_CONTAINED"
# )

# Step by Step

In [ ]:

# use composite image as test tile
asset_path = gcloud_dir # "TestDataSet/S2_comp_first.tif"
asset_path = "gs://ee-data_export/S1_mosaic_2020-01-01_2020-02-02/S1_EW_mosaic_2020-01-01_2020-02-01_tile_0.tif"
test_tif = 'S1_EW_mosaic_2020-01-01_2020-02-01_tile_0.tif'

# define id's for the STAC objects
catalog_id = "ice-shelfs-antartica_retiled"
collection_id = "sentinel-1-GRD_retiled"
item_id = "PIG"
asset_id = "HH"

We use [rioxarray](https://corteva.github.io/rioxarray) to open the GeoTIFF:

In [ ]:

source_uri = f"{asset_path}"
with rasterio.Env(GOOGLE_APPLICATION_CREDENTIALS=gcs_token):
    with rasterio.open('{}'.format(source_uri)) as src:
        print(src.profile, src.crs, src.transform)


We start by creating an `Asset` object (see [docs](https://pystac.readthedocs.io/en/latest/api.html#asset)) associated with the tile (TBD: one can also add information about the bands included in the asset through the [EO extension](https://pystac.readthedocs.io/en/latest/api.html#eo-extension)):

In [ ]:

asset_uri = source_uri

# images contain four bands: visible (B2, B3, B4) and NIR (B11)  
asset_id = "HH"# "B2-B3-B4-B11"

# create Asset object
asset = Asset(
    href=source_uri,  # link to asset
    # title=", ".join(bands.attrs['long_name']),
    title=", ".join("HH"),
    media_type=pystac.MediaType.GEOTIFF # or COG - verify e.g. with with https://github.com/rouault/cog_validator 
)

asset

### create item

We move on to create an `Item` object (see [docs](https://pystac.readthedocs.io/en/latest/api.html#item)), which will entail the asset we have previously defined (and potentially other assets). A STAC Items requires a bounding box and/or a geometry object, which we extract from the GeoTIFF. For the latter, we use `rasterio`'s `shapes` method to convert raster to vector information:  

In [ ]:




with rasterio.Env(GOOGLE_APPLICATION_CREDENTIALS=f"/Users/tud500158/.config/gcloud/legacy_credentials/{google_account}@gmail.com/adc.json"):
    with rasterio.open('{}'.format(source_uri)) as src:
        # print(src.profile, src.crs, src.transform)
        
        # Get tile bounds in WGS84, which is the standard in the GeoJSON format
        left,bottom,right,top=src.bounds         # boundingbox
        bbox = rasterio.warp.transform_bounds(src.crs, "WGS84", left, bottom, right, top )
        print(bbox)
        
        # determine footprint geometry
        poly_geom = Polygon.from_bounds(*src.bounds)
        polygons = GeoSeries(poly_geom, crs=src.crs ).to_crs("WGS84") 
        geometry = mapping(polygons) if len(polygons)>1 else mapping(polygons[0])
        print(polygons)

   

A datetime object (or a start/end datetime pair) is also required:

In [ ]:
# for a mosaic, use earliest/latest datetimes of original images
start_datetime = datetime.datetime.fromisoformat("2019-11-01")
end_datetime = datetime.datetime.fromisoformat("2020-03-01") 

In [ ]:
# create Item object
item = Item(
    id=item_id,
    geometry=geometry,
    bbox=bbox,
    datetime=None,  
    properties=dict(
        start_datetime=start_datetime.isoformat(),
        end_datetime=end_datetime.isoformat()   
    )
)

item.add_asset(asset_id, asset)
item

We then create a `Collection` object (docs [here](https://pystac.readthedocs.io/en/latest/api.html#collection)), which entails items that might be sharing some properties. Spatial and temporal extent of the collection need to be provided:  

In [ ]:
# for multiple items, determine overall extents
items = (item, )

# spatial extent
footprints = (shape(i.geometry).envelope for i in items)
collection_bbox = MultiPolygon(footprints).bounds
spatial_extent = pystac.SpatialExtent(bboxes=[collection_bbox])

# temporal extent
start = (i.properties.get('start_datetime', i.datetime) for i in items)
start = sorted(start)[0]
end = (i.properties.get('end_datetime', i.datetime) for i in items)
end = sorted(end)[-1]
temporal_extent = pystac.TemporalExtent(
    intervals=[[datetime.datetime.fromisoformat(start), 
                datetime.datetime.fromisoformat(end)]]
)

extent = pystac.Extent(spatial=spatial_extent, temporal=temporal_extent)
extent

In [ ]:
# create Collection object
collection = Collection(
    id=collection_id,
    description="Retiled Sentinel-1-GRD Images",
    extent=extent,
    license="CC-BY-SA-4.0"
)

collection.add_items(items)
collection

Finally, we add the collection to a `Catalog` object ([here](https://pystac.readthedocs.io/en/latest/api.html#catalog) are the docs), which represents the "root" of the structure:

In [ ]:
# create Catalog object
catalog = Catalog(
    id=catalog_id,
    description="Retiled Satellite Images for Antartica Ice Shelfs"
)

catalog.add_child(collection)
catalog

In [21]:
catalog.describe()

* <Catalog id=S1_mosaic>
    * <Collection id=sentinel-1-GRD_retiled>
      * <Item id=S1_EW_mosaic_2020-01-01_2020-02-01_tile_0>
      * <Item id=S1_EW_mosaic_2020-01-01_2020-02-01_tile_1>
      * <Item id=S1_EW_mosaic_2020-01-01_2020-02-01_tile_10>


We save the catalog as a self-contained object, i.e. using relative links within its elements. In this way, one can move the catalog's path or share it with other users, it will always possible to read it:  

In [ ]:
catalog_id

In [ ]:
catalog.normalize_and_save(f'./{catalog_id}', 'SELF_CONTAINED')